In [1]:
import pandas as pd
#reading the data set
df = pd.read_csv("data/data_topic.csv")

In [2]:
df['dish_reviewed'] = df['dish_reviewed'].fillna('')
df['cuisines'] = df['cuisines'].fillna('')
df['menu_item'] = df['menu_item'].fillna('')

In [3]:
def get_recommendations_test(name):
    recommended = []
    top10_list = []
    
    name = name.lower()
    #df['name'] = df['name'].str.lower()
    topic_num = df[df['name'].str.lower() == name].Topic.values
    doc_num = df[df['name'].str.lower() == name].Doc.values    
    output_df = df[df['Topic']==topic_num[0]].sort_values('Probability', ascending=False).reset_index(drop=True)

    index = output_df[output_df['Doc']==doc_num[0]].index[0]
    
    top10_list += list(output_df.iloc[index-5:index].index)
    top10_list += list(output_df.iloc[index+1:index+6].index)
    
    #output_df['name'] = output_df['name'].str.name()
    
    for each in top10_list:
        recommended.append(output_df.iloc[each]['name'])
        
    return recommended

In [4]:
get_recommendations_test('Spice Elephant')

['Southease',
 'Blend - Royal Orchid Suites',
 'Mangalore pearl - Seafood Restaurant',
 'Shanghai Court',
 'RS Shiv Sagar',
 'New Aroma',
 'Punjab Grill',
 'The Kabab Studio - Goldfinch Hotel',
 'Rustic Stove',
 'Orzuv']

In [5]:
def get_recommendations(name):
    from sklearn.metrics.pairwise import linear_kernel
    from sklearn.feature_extraction.text import TfidfVectorizer
    import pandas as pd

    recommended = []
    top30_list = []
    top30 = []
    name = name.lower()
    topic_num = df[df['name'].str.lower() == name].Topic.values
    doc_num = df[df['name'].str.lower() == name].Doc.values    
    output_df = df[df['Topic']==topic_num[0]].sort_values('Probability', ascending=False).reset_index(drop=True)
    
    
    cols = ['name','location', 'rest_type', 'rest_type', 'rest_type', 'cuisines', 'dish_reviewed', 'menu_item', 'online', 'table_booking','listed_as', 'listed_in_city']
    
    output_df['combined'] = output_df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    
    # Creating tf-idf matrix
    tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
    tfidf_matrix = tfidf.fit_transform(output_df['combined'])
    
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    scores_df = pd.DataFrame(cosine_similarities)

    index = output_df[output_df['Doc']==doc_num[0]].index[0]
    
    top30_list = list(scores_df.iloc[index].sort_values(ascending = False).iloc[1:31].index)
    
    for each in top30_list:
        if output_df.iloc[each]['name'].lower() != name:
            top30.append(output_df.iloc[each]['name'])
        
    a = [x.lower() for x in top30]

    filtered = output_df[output_df['name'].str.lower().isin(a)]
    filtered_sorted = filtered.sort_values("weighted_rating",ascending=False)
    
    for i in filtered_sorted['name']:
        if recommended.count(i) <= 0:
            recommended.append(i)
        if len(recommended) == 10:
            break

    return recommended

In [6]:
get_recommendations('Spice Elephant')

['Roots',
 'Punjab Bistro',
 'The Hidden Home',
 'Mango Tree- The Beer Garden',
 'Kaifu',
 'Gufha - The President Hotel',
 'Shanghai Court',
 'Zaica Dine & Wine',
 'Peacock',
 'Golden Rice']

In [7]:
get_recommendations('Pai Vihar')

['Sri Udupi Park',
 'Bengaluru Coffee House',
 'Gokul Kuteera',
 'Shanthi Sagar',
 'Sagar Fast Food',
 'Sri Lakshmi Vaibhav',
 'The Krishna Grand Xpress',
 'Hotel Chandrika',
 'Udupi Aatithya',
 'Srinidhi Sagar']

In [8]:
get_recommendations('Rosewood International Hotel - Bar & Restaurant')

['Citrus Cafe - Lemon Tree Hotel',
 'Wild Thyme Restaurant',
 'Kava Kitchen & Bar - Fairfield by Marriott',
 'Spice Darbar',
 'Tayo Restaurant',
 'Sangam Restaurant',
 'Mango Greens',
 'Embers - Nandhana Grand Hotel',
 'Cafe Monarch Luxur',
 'The Lounge - The Chancery Pavilion']

In [9]:
get_recommendations('Fujian Express')

['Szechuan Dragon',
 'Delicacy',
 "Chung's Chinese Corner",
 'New Peking',
 'Chung Wah',
 'Beijing Bites',
 'Mountain Spice',
 'Red Lantern',
 'Anu Momos Chinese Kitchen',
 "Kenny's Restaurant"]

In [10]:
get_recommendations('San Churro Cafe')

['Truffles',
 'Onesta',
 'Cafe Max',
 '@Italy',
 'Cafe Cassia& Deli',
 'Burp',
 'THE BEL ST.',
 'Crisp Cafe',
 'Cafe Mondo',
 'Cafe Mamma Mia']

In [11]:
get_recommendations('Woodee Pizza')

['Starbucks',
 "Domino's Pizza",
 'Ovenstory Pizza',
 'Pizza Hut',
 'Cafe Coffee Day',
 'De Oxford Cafe',
 'The Studio Cafe',
 'Cafe Hush',
 'Cuppa',
 '5th Main Cafe']

In [12]:
get_recommendations('Behrouz Biryani')

["Mani's Dum Biryani",
 "Paul's Dum Biryani",
 'Chef All Nite',
 'The Good Bowl',
 'Aramane Donne Biryani',
 'Biryani Treats',
 'Fat Chef Biryani Wale',
 'Ruchi Maayaka',
 'LSD Cafe']

In [13]:
get_recommendations('Barbeque Nation')

["AB's - Absolute Barbecues",
 'Flechazo',
 'Cafe Noir',
 'Citrus Cafe - Lemon Tree Hotel',
 'Estia - Aloft Hotel',
 'Mango Greens',
 'Ozaa',
 'The Lounge - The Chancery Pavilion',
 'Zodiac',
 "Pepper's - The Palladium"]